In [1]:
import pandas as pd
from sagemaker.sklearn.estimator import SKLearn
import time
import boto3
import json
import numpy as np
import pandas as pd
import sagemaker
from sagemaker.session import Session
from io import StringIO
import time

sess = boto3.Session()
sm = sess.client('sagemaker')
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import boto3

client = boto3.client('rds')
response = client.describe_db_instances(
    DBInstanceIdentifier='sa2-db'
)
print(response['DBInstances'][0]['Endpoint']['Address'])
hostname=response['DBInstances'][0]['Endpoint']['Address']
connection = pg.connect(user = 'postgres',
password = 'sa2dbroot',
host = str(hostname),
port = '5432',
database = 'yash_sa_schema')
#dataframe = psql.DataFrame("SELECT * FROM category", connection)
data = pd.read_sql_query('SELECT * FROM yash_sa_schema.sa_file_ingst_log ORDER BY sa_file_ingst_log.upld_ts DESC LIMIT 1',con=connection)
print(data['published_file_path'].iloc[0])
#print(data['data_src_nm'].iloc[0])
print(data['published_file_name'].iloc[0])
print(data['model'].iloc[0])
Type=data['file_upld_typ'].iloc[0]
model=data['model'].iloc[0]
DatasetPath=data['published_file_path'].iloc[0]
filename=data['published_file_name'].iloc[0]
#DatasetPath=data['DatasetPath'].iloc[0]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


sa2-db.cvl1on5n4pwi.us-east-1.rds.amazonaws.com


OperationalError: could not connect to server: Connection timed out
	Is the server running on host "sa2-db.cvl1on5n4pwi.us-east-1.rds.amazonaws.com" (34.231.138.61) and accepting
	TCP/IP connections on port 5432?


In [2]:


if model=='Arema':
    runtime = boto3.client('sagemaker-runtime')
    response = runtime.invoke_endpoint(
        EndpointName='Arema-endpoint--Banking-SA',
        Body=b'12',
        ContentType='text/csv')
    #print(response['Body'].read())
    result = (response['Body'].read())
    print(type(result))
    s = result.decode()
    #print(type(s))
    print(s[1:-1])
    y=s[1:-1]
    li = list(y.split(","))
    print(li)
    df = pd.DataFrame(li)
    df
    StringData = StringIO() 
    df.to_csv(StringData)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket_name='sa2-train-bucket',key='Output/AremaModelOutput.csv').put(Body=StringData.getvalue())
    time.sleep(10)
    df=pd.read_csv('s3://sa2-train-bucket/Output/AremaModelOutput.csv')
    df1=pd.read_csv('s3://sa2-train-bucket/Accuracy/Accuracy-Arema.csv')
    value=df1['RMSE']
    print(value)
    Final_df=df['0']-int(value)
    StringData = StringIO() 
    Final_df.to_csv(StringData)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket_name='sa2-train-bucket',key='Output/AremaModelOutputFinal.csv').put(Body=StringData.getvalue())
    #actual_values_df = pd.read_csv('AremaAccuracy.csv')
    #rmse = sqrt(mean_squared_error(actual_values.loc[:,"Volume"],y))
    #output_bucket = s3_resource.Bucket('sagemaker-us-east-1-754307369999')
    
    client = boto3.client('s3')
    obj = client.get_object(Bucket='sa2-train-bucket', Key='Accuracy/Accuracy-Arema.csv')
    model_accuracy_df1 = pd.read_csv(obj['Body'])
    original_model_rmse =  model_accuracy_df1['RMSE'].loc[0]
    print(original_model_rmse)
    if(original_model_rmse>200):
        #invoke_retrain_script
        print('retraining')
        payload3=b"""{"Run": "ok"}"""
        client = boto3.client('lambda')
        response = client.invoke(
            FunctionName="Aws-sagemaker-notebookRun",
            InvocationType='Event',
            Payload=payload3
        )
        time.sleep(5)
    else:
        print('no retraining')
        #use this model.
elif model=='lr':    
    runtime = boto3.client('sagemaker-runtime')
    data=pd.read_csv('s3://sagemaker-us-east-1-754307369999/TrainData/test_linearmodel.csv')
    response = runtime.invoke_endpoint(
        EndpointName='lr-endpoint--Banking-SA',
        Body=data.to_csv(header=False, index=False).encode('utf-8'),
        ContentType='text/csv')
    result = (response['Body'].read())
    s = result.decode()
    print(s)
    print(s[1:-1])
    y=s[1:-1]
    x=y.split(",")
    lr = []
    lr.append(x)

    print(lr)
    df = pd.DataFrame(lr)
    newDF=df.T
    StringData = StringIO() 
    newDF.to_csv(StringData)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket_name='sa2-train-bucket',key='Output/LinearOutput.csv').put(Body=StringData.getvalue())


elif model=='Churn':
    list=[]
    runtime = boto3.client('sagemaker-runtime')
    data=pd.read_csv('s3://sagemaker-us-east-1-754307369999/Input/Churntest.csv')
    response = runtime.invoke_endpoint(
        EndpointName='Churn-endpoint-ML',
        Body=data.to_csv(header=False, index=False).encode('utf-8'),
        ContentType='text/csv')
    result = json.loads(response['Body'].read().decode())
    df = pd.DataFrame(result) 
    for line in df[0]:
        y_pred = (line > 0.5)
        list.append(y_pred)
        #print(type(y_pred))
        #print(list)
        df1 = pd.DataFrame(list)
        #df1
        StringData = StringIO() 
        df1.to_csv(StringData)
        s3_resource = boto3.resource('s3')
        s3_resource.Object(bucket_name='sagemaker-us-east-1-754307369999',key='Output/ChurnOutput.csv').put(Body=StringData.getvalue())

elif model=='AremaRetrain':
    runtime = boto3.client('sagemaker-runtime')
    response = runtime.invoke_endpoint(
        EndpointName='AremaRetrain-endpoint-ML',
        Body=b'12',
        ContentType='text/csv')
    #print(response['Body'].read())
    result = (response['Body'].read())
    print(type(result))
    s = result.decode()
    #print(type(s))
    print(s[1:-1])
    y=s[1:-1]
    li = list(y.split(","))
    print(li)
    df = pd.DataFrame(li)
    StringData = StringIO() 
    df.to_csv(StringData)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket_name='sa2-train-bucket',key='Output/AremaRetrainOutput.csv').put(Body=StringData.getvalue())
elif model=='Autopilot':
    data=pd.read_csv(DatasetPath)
    runtime = boto3.client('sagemaker-runtime')
    response = runtime.invoke_endpoint(
        EndpointName="BankingSA-Endpoint"+DatasetName,
        Body=data.to_csv(header=False, index=False).encode('utf-8'),
        ContentType='text/csv')
    result = (response['Body'].read())
    lst=[]
    s = result.decode()
    for line in s:
        #print(line)
        lst.append(line)

    print(lst)
    print([s.replace('\n', '') for s in lst])
    without_empty_strings = []
    for string in lst:
        if (string != '\n'):
            without_empty_strings.append(string)


    print(without_empty_strings)
    df1 = pd.DataFrame(without_empty_strings)
    df1
    StringData = StringIO() 
    df1.to_csv(StringData)
    s3_resource = boto3.resource('s3')
s3_resource.Object(bucket_name='sa2-train-bucket',key='Output/AutopilotLoan.csv').put(Body=StringData.getvalue())



<class 'bytes'>
3482.7550466835883, 3592.0850151261375, 3554.985695062047, 3369.7419750470576, 3226.616321115141, 4611.1280415765605, 3555.551541598072, 3956.0948402270506, 4365.732186300137, 3822.328721643285, 3725.884912004805, 3977.510832439738
['3482.7550466835883', ' 3592.0850151261375', ' 3554.985695062047', ' 3369.7419750470576', ' 3226.616321115141', ' 4611.1280415765605', ' 3555.551541598072', ' 3956.0948402270506', ' 4365.732186300137', ' 3822.328721643285', ' 3725.884912004805', ' 3977.510832439738']
0    6226.018906
Name: RMSE, dtype: float64
6226.018905680518
retraining


['Y', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'N', '\n', 'N', '\n', 'N', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'N', '\n', 'N', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'N', '\n', 'N', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'N', '\n', 'N', '\n', 'Y', '\n', 'N', '\n', 'N', '\n', 'N', '\n', 'Y', '\n'

{'ResponseMetadata': {'RequestId': '3B00C5D4240DB2C8',
  'HostId': '8+tWzR7XavygALGJAqBy7nyjAukuBxivxBdoKGD458RjS0P9Cb1uBKnVxeL/J/26+6dbnJGy5KY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '8+tWzR7XavygALGJAqBy7nyjAukuBxivxBdoKGD458RjS0P9Cb1uBKnVxeL/J/26+6dbnJGy5KY=',
   'x-amz-request-id': '3B00C5D4240DB2C8',
   'date': 'Mon, 06 Apr 2020 18:55:43 GMT',
   'etag': '"b507fddeaf2eaada3e3451c29120b5d6"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"b507fddeaf2eaada3e3451c29120b5d6"'}

In [33]:
lst=[]
s = result.decode()
for line in s:
    #print(line)
    lst.append(line)
    
    
print(lst)
print([s.replace('\n', '') for s in lst])


['Y', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'N', '\n', 'N', '\n', 'N', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'N', '\n', 'N', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'N', '\n', 'N', '\n', 'Y', '\n', 'N', '\n', 'Y', '\n', 'N', '\n', 'N', '\n', 'Y', '\n', 'N', '\n', 'N', '\n', 'N', '\n', 'Y', '\n'